# IESB
# Graduação em Ciência de Dados
## Disciplina - Mineração de Dados e Inteligência Computacional
## Avaliação - 04/05/2020
## Aluno: Carlos Henrique Carneiro Costa
## Matricula: 1812120020

### Descrição
A base de dados "Home Equity" possui dados pessoas e informações de empréstimo de 5.960 empréstimos recentes. Para cada empréstimo existem 12 variáveis registradas.
A variável alvo (`BAD`) indica quando o cliente não pagou o empréstimo (valor 1), e quando ele honrou o compromisso (valor 0).

### Objetivo
Criar um modelo de ML usando RandomForest para prever se determinado cliente irá ou não honrar o empréstimo contraído.

O desempenho do modelo deve ser medido pela métrica de acurácia (`accuracy_score`).

In [ ]:
# Arquivo de dados
import os
print(os.listdir("../input"))

**Itens avaliados:**

1. Carregamento dos dados
2. Verificação para saber se os dados foram importados corretamente
3. Pré-processamento dos dados (preenchimento de valores em branco, transformação de texto em número)
4. Separar dados em treino e validação
5. Instanciar e treinar uma `RandomForest`
6. Avaliação da performance do modelo pela métrica de acurácia (`accuracy_score`).
7. Buscar a melhoria da acurácia mudando os parâmetros do modelo RandomForest.

# Bônus

Após terminar o exercício e verificar que está cumprindo todos os itens a serem avaliados, você pode fazer os itens a seguir para ganhar pontos extra na prova:

**Bônus 1:**
    
Plote uma matriz de confusão nas previsões dos dados de validação.

**Bônus 2:**
    
Explique o que é `precisão` na matriz de confusão acima e dê 1 exemplo onde isso pode ser útil.

**Bônus 3:**
    
Explique o que é `recall` na matriz de confusão acima e dê 1 exemplo onde isso pode ser útil.

> Os Bonus serão apresentados no final do Notebook

# Carregamento dos dados
Para carregar os dados corretamente devemos importar a bibiloteca pandas e saber exatemente o endereço em que se encontra o arquivo.

In [ ]:
import pandas as pd 
df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv') # arquivo deve estar entre aspas
df.head()

In [ ]:
df.info()

# Pré-processamento dos dados
Como podemos observar nas informações sobre o dataframe, a maioria dos itens das colunas são representados por números, mas as colunas REASON e JOB são do tipo objeto, sendo representadas por texto. No dataframe existem também alguns valores nulos e serão utilizadas algumas tecnicas para preenche-los. As tecnicas utilizadas em objetos se diferenciam das que usaremos para os números, pois com números apenas substituiremos por 0 os valores faltantes, mas com as objeto substituiremos pelos valores mais frequentes de cada coluna.

In [ ]:
# Quantos valores nulos temos no total
print('Soma total dos valores nulos:', df.isnull().sum().values.sum())

In [ ]:
# Valores nulos em cada coluna
df.isnull().sum()

In [ ]:
# contagem de valores na coluna REASON
df['REASON'].value_counts()

A coluna REASON tem apenas duas classificações diferentes: DebtCon e HomeImp. A que mais apareceu foi a DebtCon e vamos preenches os valores nulos da coluna REASON com DebtCon, que foi a mais frequente.

In [ ]:
df.REASON.fillna('DebtCon', inplace=True)

In [ ]:
# contagem de valores na coluna JOB
df['JOB'].value_counts()

A coluna JOB tem seis classificações diferentes. A que mais apareceu foi Other e vamos preenches os valores nulos da coluna JOB com Other, que foi a mais frequente.

In [ ]:
df.JOB.fillna('Other', inplace=True)

In [ ]:
# Substituindo os valores faltantes das colunas numéricas
df.fillna(0,inplace=True)

Veremos a diferença entre antes das tranformações e depois delas

In [ ]:
# Depois das transformações
df.head()

In [ ]:
# Vamos converter todas as variaveis categóricas para variáveis dummy
df = pd.get_dummies(df, columns=['REASON', 'JOB'])

In [ ]:
df.head()

# Separando dados em treino e validação (e teste)

In [ ]:
# Separando os dados
# Precisamos de 3 conjuntos de dados: treino, validação e teste

# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Dividindo em treino e teste a partir dos dados do dataframe
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Dividindo em treino e validação a partir dos dados de treino
train, valid = train_test_split(train, test_size=0.2, random_state=42)

train.shape, valid.shape, test.shape

In [ ]:
# Criar a lista de colunas para treino, as chamadas feats
feats = [c for c in df.columns if c not in ['BAD']]

In [ ]:
# Instanciar o modelo Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42, n_estimators = 150, min_samples_split = 4, min_samples_leaf = 4)

In [ ]:
# Treinar o modelo
rf.fit(train[feats], train['BAD'])

In [ ]:
# Previsões na base de validação
preds_val = rf.predict(valid[feats])
preds_val

In [ ]:
# Escolher a métrica para verificar o desempenho do modelo
# O exercício pede para usar a acurácia
from sklearn.metrics import accuracy_score

In [ ]:
# Medir o desempenho na base de validação com acurácia
accuracy_score(valid['BAD'], preds_val)

In [ ]:
# Previsões na base de teste
preds_teste = rf.predict(test[feats])
preds_teste

In [ ]:
# Medir o desempenho na base de teste com acurácia
accuracy_score(test['BAD'], preds_teste)

O modelo acabou desempenhando melhor nos dados de teste do que no de validação.

In [ ]:
# A biblioteca scikitplot é bastante funcinal para plotarmos dados gerados por funçoes do scikitlearn
import scikitplot as skplt

In [ ]:
# Plotando matriz de confução das variaveis preditas nos dados de validação
skplt.metrics.plot_confusion_matrix(valid['BAD'], preds_val)

In [ ]:
# Plotando matriz de confução das variaveis preditas nos dados de teste
skplt.metrics.plot_confusion_matrix(test['BAD'], preds_teste)

In [ ]:
# Para finalizar será avaliado a importancia de cada coluna em nosso modelo
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

Como podemos observar a coluna DEBTINC foi a coluna que o modelo considerou mais importante para decidir qual valor retornaria como resultado. JOB e REASON, duas colunas que fizemos mais alterações, acabaram tendo uma importância mínima.

# Precision

# Recall